In [1]:
import json
import re
import urllib.request
from datetime import datetime, timedelta
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from requests.sessions import Session
from time import sleep

In [30]:
%load_ext watermark
%watermark -adatacampus-ku -d -v --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: datacampus-ku

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

numpy   : 1.19.2
re      : 2.2.1
json    : 2.0.9
autopep8: 1.5.4



In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'
}

In [3]:
def download(url, params={}, headers=headers, method='GET', maximum=2):
    resp = request(url=url, params=params if method == 'GET' else {},
                   data=params if method == 'POST' else {},
                   headers=headers, method=method)

    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1)  # 1초?
            resp = download(url, params, headers, method,
                            maximum-1)  # inf loop
        else:
            print(e)

    return resp

In [4]:
def cleaning(data):
    data = re.sub(r'오류를 우회하기 위한 함수 추가', '', data)
    # 안지워질 때를 대비
    data = re.sub(r'\b[a-z_][a-zA-Z_()]{4,}\b', '', data)
    # 1. 사이드 스페이스가 있을 때 반드시 소문자+_  1글자로 시작, 영문(대소문자 포함) 4글자 이상 삭제 => 제일 뒷줄의 javascript 제거
    data = re.sub(r'\b[(){}/]{2,}', '', data)
    # 2. 함수 기호 삭제 => () {} 제거
    data = re.sub(r'([.,?!])(\w)', '\g<1> \g<2>', data)
    # 3. 구두점 다음에 글자가 있는 경우 공백 추가 => .단어 => . 단어로 변경
    data = re.sub(r'\s{2,}', '\n', data).strip()
    # 4. 공백 2개 이상 반복되는 경우 한 개만 => \n\n\n\n\n 또는      =>\n 또는
    data = re.sub(r'[^\s\w\d.,?!:\'\"]', '', data)
    # 5. 공백, 문자, 숫자, 구두점을 제외한 나머지 글자 삭제 => 모든 특수문자 제거
    data = '\n'.join(data.splitlines()[1:])
    # * 네이버에서 수집한 뉴스는 첫 줄에 주석이 달려있음 => 첫 줄 주석 삭제
    return data

In [5]:
def download_video(start,param,broadcast):
    news_url = ''.join([text_url, '?sid1=', broadcast, '&oid=', re.findall(r'([\d]*)_', param)[0],
                        '&aid=', re.findall(r'_([\d]*)', param)[0]])
    print(news_url)
    news = download(news_url)
    dom = BeautifulSoup(news.text, 'lxml')
    
    #뉴스동영상이 iframe 안에 존재함, iframe 단으로 내려가기
    vid_url="https://news.naver.com/main/readVod.naver?&oid={}&aid={}".format(re.findall(r'([\d]*)_', param)[0],re.findall(r'_([\d]*)', param)[0])
    headers_video={
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36',
    'Referer' : news_url #레퍼러 없으면 불러와 지지 않음
    }
    vid_page=BeautifulSoup(request(url=vid_url,headers=headers_video,method='POST').text,'lxml')
    
    #iframe  단 안에 존재하는 video id와 key 값 찾기
    vid_temp=vid_page.find_all(attrs={'type':'text/javascript'})[1]
    vid_id=re.findall("(?<=\(\\\')([A-Za-z0-9_-]*)(?=\\\')",vid_temp.text)[0]
    vid_key=re.findall("(?<=\\\'value\\\'\:\\')([A-Za-z0-9_-]*)(?=\\\')",vid_temp.text)[0]
    
    #동기방식으로 이루어진 영상 불러오기 방식
    vid_download_url="https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0/"+vid_id
    vid_resp=download(vid_download_url,{"key": vid_key},headers_video)
    video=json.dumps(vid_resp.json()['videos'])
    
    #최대화질로 다운로드
    if re.search(r'"720P".+?"source": "(.+)",', video)!=None:
        video_link = re.findall(r'"720P".+?"source": "(.+)",', video)[0]
    elif re.search(r'"480P".+?"source": "(.+)",', video)!=None:
        video_link = re.findall(r'"480P".+?"source": "(.+)",', video)[0]
    elif re.search(r'"360P".+?"source": "(.+)",', video)!=None:
        video_link = re.findall(r'"360P".+?"source": "(.+)",', video)[0]
    else:
        video_link = re.findall(r'"270P".+?"source": "(.+)",', video)[0]
        
    save_file = './MOV/{}.mp4'.format(re.sub(r'\W', '', news_url.split('/')[-1]))
    urllib.request.urlretrieve(video_link, save_file)        

In [131]:
def download_text(start, param, broadcast):
    news_url = ''.join([text_url, '?sid1=', broadcast, '&oid=', re.findall(r'([\d]*)_', param)[0],
                        '&aid=', re.findall(r'_([\d]*)', param)[0]])
    print(news_url)
    news = download(news_url)
    dom = BeautifulSoup(news.text, 'lxml')
    main_area = dom.select_one('#main_content')
    file_name = re.sub(r'\W', '', news_url.split('/')[-1])
    if main_area is not None:
        article_area = dom.select_one('#articleBodyContents')
        with open('TXT/'+file_name+'.txt', 'w', encoding='utf-8') as f:  # TXT 폴더에 저장
            f.write(cleaning(article_area.text))
    else:
        article_area = dom.select_one('#newsEndContents')
        # 다른 cleaning 사용
        with open('TXT/'+file_name+'.txt', 'w', encoding='utf-8') as f:  # TXT 폴더에 저장
            f.write(re.sub(r'\s{2,}', '\n', re.findall(r'displayRMCPlayer\(\)\;(.+)', re.sub(r'[\n\t]', '', article.text))[0]))

In [132]:
# 다른 html 구조에 대한 확인
# 이란 뭐시기 뉴스 .. https://sports.news.naver.com/news?oid=056&aid=00000036083
download_text('20130615', '056_0000003608', '355')

https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003608


In [111]:
def download_news(start, param, broadcast):
    download_video(start, param, broadcast)
    download_text(start, param, broadcast)
    return True

In [112]:
# 방송사별 가장 첫 뉴스 주소
news_list = {
    'KBS': 'https://news.naver.com/main/tvgallery/replay.naver?sid=355&date=20130613',
    'MBC': 'https://news.naver.com/main/tvgallery/replay.naver?sid=370&date=201300101',
    'SBS': 'https://news.naver.com/main/tvgallery/replay.naver?sid=371&date=201300101',
    'JTBC': 'https://news.naver.com/main/tvgallery/replay.naver?sid=742&date=20131014',
    'TVC': 'https://news.naver.com/main/tvgallery/replay.naver?sid=750&date=20131125',
    'YTN': 'https://news.naver.com/main/tvgallery/replay.naver?sid=751&date=20131121',
    'CHA': 'https://news.naver.com/main/tvgallery/replay.naver?sid=74b&date=20131125',
    'MBN': 'https://news.naver.com/main/tvgallery/replay.naver?sid=74e&date=20131125',
    'NESY': 'https://news.naver.com/main/tvgallery/replay.naver?sid=74f&date=20140106',
}

### KBS 뉴스 출력

In [113]:
base_url = 'https://news.naver.com/main/tvgallery/replay.naver?sid=355&date='
text_url = 'https://news.naver.com/main/read.naver'
date = '20130613'
broadcast = '355'

In [10]:
start='https://news.naver.com/main/tvgallery/replay.naver?sid=355&date=20130613'
n=1
while(date != '20210725'):
    resp = download(''.join([base_url, date]))
    nlist = re.findall('<a href="#([0-9_]+)"', resp.text)
    # 맨 앞과 맨 마지막 뉴스 제외
    for _ in nlist[1:-1]:
        print(_)
        download_news(start, _, broadcast)    
    date = (datetime.strptime(date, '%Y%m%d') + timedelta(days=1)).strftime('%Y%m%d')
    start = ''.join(['https://news.naver.com/main/tvgallery/replay.naver?sid=355&date=', date])
    n+=1

056_0000003463
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003463
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003463
056_0000003464
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003464
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003464
056_0000003465
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003465
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003465
056_0000003466
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003466
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003466
056_0000003467
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003467
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003467
056_0000003468
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003468
https://news.naver.com/main/read.naver?sid1=355&oid=056&aid=0000003468
056_0000003469
https://news.naver.com/main/read.naver?sid1

KeyboardInterrupt: 